In [6]:
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300
from collections import Counter
from tqdm import tqdm
import pandas as pd
import numpy as np
import sys
import os

# Define root directory
# root = os.path.dirname(os.path.abspath(__file__))
root = "."
sys.path.append(os.path.join(root, "..", "src"))
from default import CONFIGPATH

ChEMBL_ALL = pd.read_csv(os.path.join(root, "..", "config", "chembl_processed", "activities_all_raw.csv"), low_memory=False)
print(len(ChEMBL_ALL[(ChEMBL_ALL['assay_chembl_id'] == "CHEMBL1909088") & (ChEMBL_ALL['standard_type'] == "IC50") & (ChEMBL_ALL['canonical_smiles'].isna())]))
del ChEMBL_ALL

# Loading ChEMBL preprocessed data
print("Loading ChEMBL preprocessed data...")
ChEMBL = pd.read_csv(os.path.join(root, "..", "config", "chembl_processed", "activities_preprocessed.csv"), low_memory=False)

36
Loading ChEMBL preprocessed data...


,activity_id,assay_id,assay_chembl_id,assay_type,assay_confidence_score,assay_organism,doc_id,tid,target_type,target_organism,...,canonical_smiles,MW,standard_relation,standard_value,standard_units,standard_type,activity_comment,pchembl_value,standard_text_value,bao_endpoint
6879236,7613158,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,=,884.0,nM,IC50,NaN,6.05,NaN,BAO_0000190
6884567,7618477,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,=,4556.0,nM,IC50,NaN,5.34,NaN,BAO_0000190
6887567,7621466,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,=,6918.0,nM,IC50,NaN,5.16,NaN,BAO_0000190
6910968,7644829,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,NaN,NaN,NaN,IC50,Inhibition < 50% @ 10 uM and thus dose-reponse...,NaN,Not Active,BAO_0000190
6928204,7662042,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,NaN,NaN,NaN,IC50,Inhibition < 50% @ 10 uM and thus dose-reponse...,NaN,Not Active,BAO_0000190
6932491,7666322,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,NaN,NaN,NaN,IC50,Inhibition < 50% @ 10 uM and thus dose-reponse...,NaN,Not Active,BAO_0000190
6936444,7670262,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,NaN,NaN,NaN,IC50,Inhibition < 50% @ 10 uM and thus dose-reponse...,NaN,Not Active,BAO_0000190
6944514,7678317,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,NaN,NaN,NaN,IC50,Inhibition < 50% @ 10 uM and thus dose-reponse...,NaN,Not Active,BAO_0000190
6954855,7688638,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,NaN,NaN,NaN,IC50,Inhibition < 50% @ 10 uM and thus dose-reponse...,NaN,Not Active,BAO_0000190
6957424,7691200,774699,CHEMBL1909088,B,8,NaN,59306,52,SINGLE PROTEIN,Homo sapiens,...,NaN,NaN,NaN,NaN,NaN,IC50,Inhibition < 50% @ 10 uM and thus dose-reponse...,NaN,Not Active,BAO_0000190


In [ ]:
print(f"Number of activities: {len(ChEMBL)}")
print(f"Number of activities with non nan values: {len(ChEMBL[ChEMBL['value'].isna() == False])}")
print(f"Number of unique compounds: {len(set(ChEMBL['compound_chembl_id']))}")
print(f"Number of unique assays: {len(set(ChEMBL['assay_chembl_id']))}")
print(f'Assay types: {Counter(ChEMBL[["assay_chembl_id", "assay_type"]].drop_duplicates(subset="assay_chembl_id")["assay_type"])}')
print(f"Number of unique targets: {len(set(ChEMBL['target_chembl_id']))}")
print(f'Target types: {Counter(ChEMBL[["target_chembl_id", "target_type"]].drop_duplicates(subset="target_chembl_id")["target_type"])}')
print(f"Relations: {Counter(ChEMBL['relation'])}")
print(f"Original pChEMBL: {len(ChEMBL[ChEMBL['pchembl'].isna() == False])}")
print(f"Calculated pChEMBL: {len(ChEMBL[ChEMBL['pchembl_calculated'].isna() == False])}")

thr = 0.01
ChEMBL_filtered = ChEMBL[(ChEMBL['pchembl'].isna() == False) & (ChEMBL['pchembl_calculated'].isna() == False)].copy()
ChEMBL_filtered['pchembl'] = ChEMBL_filtered['pchembl'].clip(lower=1, upper=9)
ChEMBL_filtered = ChEMBL_filtered[["pchembl", "pchembl_calculated"]]
ChEMBL_filtered['difference'] = [np.abs(i-j) for i,j in zip(ChEMBL_filtered['pchembl'], ChEMBL_filtered['pchembl_calculated'])]
perc = len(ChEMBL_filtered[ChEMBL_filtered['difference'] < thr]) * 100 / len(ChEMBL_filtered)
print(f"Percentage of pChEMBL original vs calculated having difference < {thr}: {perc}")

In [ ]:
LABEL = 'activity_type'
print(f"Number of standard activity types: {len(set(ChEMBL[LABEL]))}")
s = ChEMBL[[LABEL]].astype("string").fillna("")
out = (s.value_counts(dropna=False).reset_index(name="count").sort_values("count", ascending=False, ignore_index=True))
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)
print(f"100 types coverage: {out['cumulative_prop'].tolist()[100]}")

In [ ]:
LABEL = 'unit'
print(f"Number of units: {len(set(ChEMBL[LABEL]))}")
s = ChEMBL[[LABEL]].astype("string").fillna("")
out = (s.value_counts(dropna=False).reset_index(name="count").sort_values("count", ascending=False, ignore_index=True))
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)
print(f"100 units coverage: {out['cumulative_prop'].tolist()[100]}")

In [ ]:
LABEL = ['activity_type', 'unit']
s = pd.DataFrame([str(i) + " -- " + str(j) for i,j in zip(ChEMBL[LABEL[0]], ChEMBL[LABEL[1]])])
out = (s.value_counts(dropna=False).reset_index(name="count").sort_values("count", ascending=False, ignore_index=True))
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)
print(f"Number of act-unit pairs: {len(out)}")
print(f"100 pairs coverage: {out['cumulative_prop'].tolist()[100]}")

In [ ]:
s = ChEMBL[["activity_type", "unit"]].astype("string").fillna("")
out = (
s.value_counts(subset=["activity_type", "unit"], dropna=False)
    .reset_index(name="count")
    .sort_values("count", ascending=False, ignore_index=True)
)
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)
print(f"Number of activity entries having no associated unit: {sum(out[out['unit'] == '']['count'])}")

In [ ]:
ChEMBL[(ChEMBL['unit'].isna()) & (ChEMBL['pchembl'].isna() == False)]

In [ ]:
filtered = ChEMBL[(ChEMBL['activity_type'] == 'IC50') & (ChEMBL['unit'].isna())]
out = (
filtered.value_counts(subset=["doc_chembl_id"], dropna=False)
    .reset_index(name="count")
    .sort_values("count", ascending=False, ignore_index=True)
)
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)

In [ ]:
out

In [ ]:
filtered[filtered['assay_chembl_id'] == "CHEMBL1909088"]

In [ ]:
filtered = ChEMBL[(ChEMBL['activity_type'] == 'ACTIVITY') & (ChEMBL['unit'].isna())]
out = (
filtered.value_counts(subset=["doc_chembl_id"], dropna=False)
    .reset_index(name="count")
    .sort_values("count", ascending=False, ignore_index=True)
)
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)

In [ ]:
### FILTERING FOR PATHOGENS ###

def get_pathogen_code(pathogen):
    return str(pathogen.split()[0][0] + pathogen.split()[1]).lower() if len(pathogen.split()) > 1 else pathogen.lower()

In [ ]:
# List of pathogens
pathogens = ["Acinetobacter baumannii", "Candida albicans", "Campylobacter", "Escherichia coli", "Enterococcus faecium", "Enterobacter",
             "Helicobacter pylori", "Klebsiella pneumoniae", "Mycobacterium tuberculosis", "Neisseria gonorrhoeae", "Pseudomonas aeruginosa",
             "Plasmodium falciparum", "Staphylococcus aureus", "Schistosoma mansoni", "Streptococcus pneumoniae"]

RESULTS = {}
# For each pathogen
pathogens = np.array(pathogens)
for pathogen in pathogens:

    print(f"Filtering for pathogen: {pathogen}...")
    pathogen_code = get_pathogen_code(pathogen)
    pathogen_data = ChEMBL[ChEMBL['target_organism'].str.contains(pathogen, case=False, na=False) | 
                    ChEMBL['assay_organism'].str.contains(pathogen, case=False, na=False)].reset_index(drop=True)

    print(f"Number of activities: {len(pathogen_data)}")
    print(f"Number of unique compounds: {len(set(pathogen_data['compound_chembl_id']))}")
    RESULTS[pathogen_code] = [len(pathogen_data), len(set(pathogen_data['compound_chembl_id']))]
    print("\n")

In [ ]:
plt.figure(figsize=(8, 4))

x = [i for i in range(len(pathogens))]
inds = np.argsort([RESULTS[get_pathogen_code(i)][1] for i in pathogens])[::-1]
plt.bar(x, [RESULTS[get_pathogen_code(i)][0] for i in pathogens[inds]], zorder=2, ec='k', color="#AA96FA", label='Activities')
plt.bar(x, [RESULTS[get_pathogen_code(i)][1] for i in pathogens[inds]], zorder=2, ec='k', color="#FAD782", label='Compounds')

plt.ylim([10 ** 2, 2*10 ** 6])
plt.yscale('log')
plt.ylabel("Number of")
plt.xticks(x, [get_pathogen_code(i) for i in pathogens[inds]], rotation=45, ha='right')
plt.grid(linestyle='--')
plt.legend(loc='upper right', framealpha=1, edgecolor='k', prop={'size': 9})
plt.show()

In [ ]:
# Filtering data for pathogens
pathogen_data = ChEMBL[ChEMBL['target_organism'].str.lower().isin([p.lower() for p in pathogens]) | 
                       ChEMBL['assay_organism'].str.lower().isin([p.lower() for p in pathogens])].reset_index(drop=True)

In [ ]:
print(f"Number of activities: {len(pathogen_data)}")
print(f"Number of activities with non nan values: {len(pathogen_data[pathogen_data['value'].isna() == False])}")
print(f"Number of unique compounds: {len(set(pathogen_data['compound_chembl_id']))}")
print(f"Number of unique assays: {len(set(pathogen_data['assay_chembl_id']))}")
print(f'Assay types: {Counter(pathogen_data[["assay_chembl_id", "assay_type"]].drop_duplicates(subset="assay_chembl_id")["assay_type"])}')
print(f"Number of unique targets: {len(set(pathogen_data['target_chembl_id']))}")
print(f'Target types: {Counter(pathogen_data[["target_chembl_id", "target_type"]].drop_duplicates(subset="target_chembl_id")["target_type"])}')
print(f"Relations: {Counter(pathogen_data['relation'])}")
print(f"Original pChEMBL: {len(pathogen_data[pathogen_data['pchembl'].isna() == False])}")
print(f"Calculated pChEMBL: {len(pathogen_data[pathogen_data['pchembl_calculated'].isna() == False])}")

thr = 0.01
pathogen_data_filtered = pathogen_data[(pathogen_data['pchembl'].isna() == False) & (pathogen_data['pchembl_calculated'].isna() == False)].copy()
pathogen_data_filtered['pchembl'] = pathogen_data_filtered['pchembl'].clip(lower=1, upper=9)
pathogen_data_filtered = pathogen_data_filtered[["pchembl", "pchembl_calculated"]]
pathogen_data_filtered['difference'] = [np.abs(i-j) for i,j in zip(pathogen_data_filtered['pchembl'], pathogen_data_filtered['pchembl_calculated'])]
perc = len(pathogen_data_filtered[pathogen_data_filtered['difference'] < thr]) * 100 / len(pathogen_data_filtered)
print(f"Percentage of pChEMBL original vs calculated having difference < {thr}: {perc}")

In [ ]:
LABEL = 'activity_type'
print(f"Number of standard activity types: {len(set(pathogen_data[LABEL]))}")
s = pathogen_data[[LABEL]].astype("string").fillna("")
out = (s.value_counts(dropna=False).reset_index(name="count").sort_values("count", ascending=False, ignore_index=True))
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)
print(f"100 types coverage: {out['cumulative_prop'].tolist()[100]}")

In [ ]:
LABEL = 'unit'
print(f"Number of units: {len(set(pathogen_data[LABEL]))}")
s = pathogen_data[[LABEL]].astype("string").fillna("")
out = (s.value_counts(dropna=False).reset_index(name="count").sort_values("count", ascending=False, ignore_index=True))
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)
print(f"100 units coverage: {out['cumulative_prop'].tolist()[100]}")

In [ ]:
LABEL = ['activity_type', 'unit']
s = pd.DataFrame([str(i) + " -- " + str(j) for i,j in zip(pathogen_data[LABEL[0]], pathogen_data[LABEL[1]])])
out = (s.value_counts(dropna=False).reset_index(name="count").sort_values("count", ascending=False, ignore_index=True))
total_count = out['count'].sum()
out['cumulative_prop'] = (out['count'].cumsum() / total_count).round(3)
print(f"Number of act-unit pairs: {len(out)}")
print(f"100 pairs coverage: {out['cumulative_prop'].tolist()[100]}")

In [ ]:
out[:10]

In [ ]:
# From Assay ChEMBL ID to number of nans / non-nans
PATHOGEN_ASSAYS = set(pathogen_data['assay_chembl_id'])
PATHOGEN_ASSAYS = {i: {'nans': set(), 'non-nans': set()} for i in PATHOGEN_ASSAYS}

# Collect info
for activity_id, assay_chembl_id, value in tqdm(zip(pathogen_data['activity_id'], pathogen_data['assay_chembl_id'], pathogen_data['value'])):
    if np.isnan(value):
        PATHOGEN_ASSAYS[assay_chembl_id]['nans'].add(activity_id)
    else:
        PATHOGEN_ASSAYS[assay_chembl_id]['non-nans'].add(activity_id)

# Assays with at least one nan
assays_nan_values = Counter(pathogen_data[pathogen_data['value'].isna()]['assay_chembl_id'])
print(f"Number of assays with at least one nan value: {len(assays_nan_values)}")
assert len(assays_nan_values) == len([i for i in PATHOGEN_ASSAYS if len(PATHOGEN_ASSAYS[i]['nans']) > 0])

In [ ]:
# Calculate proportion of nans
proportion_nans = {}
for assay in sorted(PATHOGEN_ASSAYS):
    if len(PATHOGEN_ASSAYS[assay]['nans']) > 0:
        proportion_nans[assay] = len(PATHOGEN_ASSAYS[assay]['nans']) / (len(PATHOGEN_ASSAYS[assay]['nans']) + len(PATHOGEN_ASSAYS[assay]['non-nans']))

print(len(proportion_nans))

In [ ]:
len([i for i in proportion_nans if (len(PATHOGEN_ASSAYS[i]['nans']) + len (PATHOGEN_ASSAYS[i]['non-nans'])) == 1])

In [ ]:
len([i for i in proportion_nans if (len(PATHOGEN_ASSAYS[i]['nans']) + len (PATHOGEN_ASSAYS[i]['non-nans'])) > 1 and proportion_nans[i] > 0.9])

In [ ]:
# pathogen_data[pathogen_data['assay_chembl_id'] == "CHEMBL1003929"]